In [1]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
from geopy import distance
import numpy as np

In [2]:
geolocator = Nominatim(user_agent="test_tk")

In [4]:
bigdf = pd.read_csv('./results/5_years_merged_with_weather_and_311and_policedistance.csv')

In [7]:
POLICE_DF = pd.read_csv('./Police_Stations_20240120.csv')

In [32]:
def find_closest(lat, long):
    if lat == 0 or long == 0:
        return 'No Lat/Long', 999999
    else:
        try:
            input_latlong = (long,lat)
            #distance_list = []
            pdf = POLICE_DF.copy()
            pdf['distance'] = pdf.apply(lambda x: distance.distance(input_latlong, (x.LONGITUDE,x.LATITUDE)).miles, axis=1)
            nearest_station = pdf.iloc[pdf['distance'].idxmin()]
            #print(nearest_station)
            return nearest_station['ADDRESS'], nearest_station['distance']
            """for row in POLICE_DF.iterrows():
                distance_holder = {}
                station_latlong = [row['LATITUDE'], row['LONGITUDE']]
                distance_holder['station'] = row['ADDRESS']
                distance_holder['distance'] = distance.distance(input_latlong, station_latlong).miles

                distance_list.append(distance_holder)
                distance_df = pd.DataFrame(distance_list)
                nearest_station = distance_df.iloc[distance_df['distance'].idxmin()]
                print(f"{distance_df.head()}")
                return nearest_station['station'], nearest_station['distance'] """
        except Exception as e:
            #print(e)
            return 'Error something something', 999999

In [33]:
find_closest(41.818171,-87.634761)

('5101 S Wentworth Ave', 0.2952944847352142)

In [37]:
minidf = bigdf.iloc[50:400]
minidf[['closest_station', 'closest_station_distance']] = minidf.apply(lambda x: find_closest(x.latitude, x.longitude), axis=1, result_type='expand')

C:\Users\tkkim\AppData\Local\Temp\ipykernel_25720\3486701180.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minidf[['closest_station', 'closest_station_distance']] = minidf.apply(lambda x: find_closest(x.latitude, x.longitude), axis=1, result_type='expand')


In [34]:
#bigdf[['closest_station', 'closest_station_distance']] = bigdf[bigdf['latitude'].isnull() == False].apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_type='expand')

In [39]:
bigdf[['closest_station', 'closest_station_distance']] = bigdf.apply(lambda x: find_closest(x.latitude, x.longitude), axis=1, result_type='expand')

c:\Users\tkkim\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)
c:\Users\tkkim\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)
c:\Users\tkkim\AppData\Local\Programs\Python\Python311\Lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization h

In [41]:
bigdf.to_csv('./results/5_years_merged_with_weather_and_311and_policedistance.csv', index=False)

In [40]:
bigdf['closest_station'].value_counts()

closest_station
4650 N Pulaski Rd            126155
3510 S Michigan Ave           91850
2255 E 103rd St               86124
6464 N Clark St               66840
5400 N Lincoln Ave            64612
5101 S Wentworth Ave          63675
5555 W Grand Ave              62736
5701 W Madison St             59033
3315 W Ogden Ave              54965
7040 S Cottage Grove Ave      54771
1412 S Blue Island Ave\n      48172
1438 W 63rd St                44980
1718 S State St               43696
727 E 111th St                40676
2150 N California Ave         38913
7808 S Halsted St             37980
1900 W Monterey Ave           35332
1160 N Larrabee St            34239
3151 W Harrison St            34134
850 W Addison St              32129
3120 S Halsted St             24341
Error something something     17011
3420 W 63rd St                14446
5151 N Milwaukee Ave           4329
Name: count, dtype: int64